In [1]:
import pandas as pd
import random
import os
import copy
import numpy as np
import json
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import seaborn as sns
from functools import reduce
#from constants import diffMappingToScore, questions
#from utils import read_json

In [12]:
from constants import diffMappingToScore, questions

In [2]:
#####################
# This notebook computes the following measures
#
#  - Comprehension accuracy
#  - Comprehension efficiency (response time)
#  - Perceived difficulty
#
#######################

In [3]:
def addQuestionInfo(allData,questions):
    
    #change the type of questionID to integer
    allData['questionID'] = allData['questionID'].astype('int')
    
    #extend the columns of questionnaireData with those in DataFrame(questions) based the common question ID
    allData = allData.merge(pd.DataFrame(questions), left_on=['questionID'], right_on=['id'])
    
    return allData

In [4]:
def read_json(path):
    with open(path, encoding='utf-8') as inFile:
        return json.load(inFile)
    

def answeringAccuracy(allData,grouper):
    
    #compute answer accuracy as 1 if x['questionAnswer']==x['ExpectedAnswer'] else 0
    allData["AnswerAccuracy"] = allData.apply(lambda x: 1 if x['questionAnswer']==x['ExpectedAnswer'] else 0, axis=1)
    
    #group by grouper and compute mean AnswerAccuracy
    allData = allData.groupby(grouper,as_index=False).agg({'AnswerAccuracy':'mean'})
    
    return allData


def timeInterval(x):
    return x.iloc[-1]-x.iloc[0]

def responseTime(allData):
    
    #goup by questionID and Typ1, Type2 and Type3
    #Grouping by questionID allows us to compute timeInterval() by substracting the last timestamp for the question from the first one
    #Grouping by 'Type1','Type2','Type3' is done just to keep track of these attributes for further grouping. The grouping by questionID is already at the most fine-grained question level
    allData = allData.groupby(['questionID','participant','Type1','Type2','Type3'],as_index=False).agg(timeInterval=('questionTimestamp', timeInterval))

    return allData


def diffToScore(x):
    return diffMappingToScore[x]

def perceivedDifficulty(allData,grouper):
    
    allData['difficultyScore'] = allData.apply(lambda x: diffToScore(x['questionAnswer']),axis=1)
    
    #group by grouper
    allData = allData.groupby(grouper,as_index=False).agg({'difficultyScore':'mean'})

    return allData


In [5]:
#######
#
# Loading and Preprocessing 
#
######

In [8]:
#loading the data
directory = "data/"
pattern = re.compile("EyeMind_EyeMind_(.+?)_collected-data.json")
allData = None


for subdir, dirs, files in os.walk(directory):
    for file in files:  
        if pattern.match(file):
            filename = os.path.join(subdir, file)
            print(filename)
            data = pd.DataFrame(read_json(filename)["processedGazeData"]["gazeData"])
           
            data["participant"] = re.search('data/(.+?)\\\EyeMind_EyeMind_',filename).group(1) #for windows
            #data["participant"] = re.search('data/(.+?)/EyeMind_EyeMind_',filename).group(1) # for mac
            allData = pd.concat([allData,data],axis=0)
            
allData = allData.reset_index(drop=True)

data/KP1-no\EyeMind_EyeMind_1674478819228_collected-data.json
data/KP10-no\EyeMind_EyeMind_1674639630314_collected-data.json
data/KP11-no\EyeMind_EyeMind_1674643867484_collected-data.json
data/KP12-no\EyeMind_EyeMind_1674655397191_collected-data.json
data/KP13-no\EyeMind_EyeMind_1674659668246_collected-data.json
data/KP14-no\EyeMind_EyeMind_1674664209767_collected-data.json
data/KP15-no\EyeMind_EyeMind_1674668667713_collected-data.json
data/KP16-no\EyeMind_EyeMind_1674721956398_collected-data.json
data/KP17-no\EyeMind_EyeMind_1674725603802_collected-data.json
data/KP18-no\EyeMind_EyeMind_1674731371870_collected-data.json
data/KP19-no\EyeMind_EyeMind_1674737722501_collected-data.json
data/KP2-no\EyeMind_EyeMind_1674487735527_collected-data.json
data/KP20-no\EyeMind_EyeMind_1674747102905_collected-data.json
data/KP21-no\EyeMind_EyeMind_1674806583785_collected-data.json
data/KP22-no\EyeMind_EyeMind_1674811515606_collected-data.json
data/KP23-no\EyeMind_EyeMind_1674817301486_collected-data

In [9]:
#previewing the data
allData.head()

,questionTimestamp,questionEventType,questionText,questionAnswer,questionPosition,questionID,eventSource,Timestamp,validLeft,validRight,...,leftY,rightX,rightY,x,y,leftDistance,rightDistance,tabName,element,participant
0,1.674477e+12,questionOnset,"The activities ""Register arrival of container...",,0.0,1,questionnaire,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,KP1-no
1,NaN,None,None,None,NaN,None,eye-tracker,5307403.202,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,KP1-no
2,NaN,None,None,None,NaN,None,eye-tracker,5307411.529,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,KP1-no
3,NaN,None,None,None,NaN,None,eye-tracker,5307419.884,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,KP1-no
4,NaN,None,None,None,NaN,None,eye-tracker,5307428.198,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,KP1-no


In [10]:
allData["participant"].unique()

array(['KP1-no', 'KP10-no', 'KP11-no', 'KP12-no', 'KP13-no', 'KP14-no',
       'KP15-no', 'KP16-no', 'KP17-no', 'KP18-no', 'KP19-no', 'KP2-no',
       'KP20-no', 'KP21-no', 'KP22-no', 'KP23-no', 'KP24-no', 'KP3-no',
       'KP4-no', 'KP5-no', 'KP6-no', 'KP7-no', 'KP8-no', 'KP9-no',
       'SP1-no', 'SP10-no', 'SP11-no', 'SP12-no', 'SP13-no', 'SP14-no',
       'SP15-no', 'SP16-no', 'SP17-no', 'SP18-no', 'SP19-no', 'SP2-no',
       'SP20-no', 'SP21-no', 'SP22-no', 'SP3-no', 'SP4-no', 'SP5-no',
       'SP6-no', 'SP7-no', 'SP8-no', 'SP9-no'], dtype=object)

In [ ]:
#######################
#
# Comprehension Accuracy
#
#######################

In [ ]:
#measure specific pre-processing

In [13]:
# select rows with questionEventType='questionOffset' and 'questionID' is in the questions considered for the analysis
ansAccData = allData.loc[(allData['questionEventType']=='questionOffset') & (allData['questionID'].isin([str(question["id"]) for question in questions]))].copy(deep=True)

#add question info
ansAccData = addQuestionInfo(ansAccData,questions)

#Select control-flow question type
ansAccData = ansAccData.loc[(ansAccData['Type2'] == 'Control-flow')] 

In [14]:
"""
2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
"""
ansAccData = ansAccData.drop(ansAccData[(allData['participant'] == 'SP11-no') & (ansAccData['Type3'] == 'Exclusiveness')].index)
#ansAcc_new.shape

C:\Users\la1949\AppData\Local\Temp\ipykernel_18452\853378133.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ansAccData = ansAccData.drop(ansAccData[(allData['participant'] == 'SP11-no') & (ansAccData['Type3'] == 'Exclusiveness')].index)


In [ ]:
##Check for nan values
#print(ansAcc[ansAcc.isna().any(axis=1)])

In [15]:
# Example query: For each participant, we obtain one aggregated score for local questions and one aggregate score for global questions
answeringAccuracy(ansAccData,['participant','Type1'])

,participant,Type1,AnswerAccuracy
0,KP1-no,Global,0.75
1,KP1-no,Local,1.00
2,KP10-no,Global,1.00
3,KP10-no,Local,1.00
4,KP11-no,Global,0.25
...,...,...,...
87,SP7-no,Local,1.00
88,SP8-no,Global,0.75
89,SP8-no,Local,1.00
90,SP9-no,Global,0.25


In [110]:
#Check whether answer accuracy is always lower for global than for local tasks for each participant on average
answeringAccuracyCompare = answeringAccuracy(ansAccData,['participant','Type1'])
answeringAccuracyCompare_G = answeringAccuracyCompare.loc[answeringAccuracyCompare['Type1'] == 'Global']
answeringAccuracyCompare_L = answeringAccuracyCompare.loc[answeringAccuracyCompare['Type1'] == 'Local']
answeringAccuracyCompare_G_L = pd.merge(answeringAccuracyCompare_G, answeringAccuracyCompare_L, on='participant')

answeringAccuracyCompare_G_L['Glo > Loc'] = answeringAccuracyCompare_G_L.apply(lambda x: True if x['AnswerAccuracy_x'] < x['AnswerAccuracy_y'] else (x['AnswerAccuracy_x'],x['AnswerAccuracy_y']), axis=1)
answeringAccuracyCompare_G_L

,participant,Type1_x,AnswerAccuracy_x,Type1_y,AnswerAccuracy_y,Glo > Loc
0,KP1-no,Global,0.75,Local,1.00,True
1,KP10-no,Global,1.00,Local,1.00,"(1.0, 1.0)"
2,KP11-no,Global,0.25,Local,0.75,True
3,KP12-no,Global,1.00,Local,1.00,"(1.0, 1.0)"
4,KP13-no,Global,0.50,Local,1.00,True
5,KP14-no,Global,1.00,Local,1.00,"(1.0, 1.0)"
6,KP15-no,Global,1.00,Local,1.00,"(1.0, 1.0)"
7,KP16-no,Global,0.75,Local,0.75,"(0.75, 0.75)"
8,KP17-no,Global,0.50,Local,0.75,True
9,KP18-no,Global,0.75,Local,1.00,True


In [16]:
# Descriptives

In [17]:
#Calculate mean per participant
ansAcc_participant = answeringAccuracy(ansAccData,['participant','Type1', 'Type2'])
#Calculate descriptives - mean
ansAcc_participant.groupby('Type1').agg({'AnswerAccuracy':'mean'})

,AnswerAccuracy
Type1,
Global,0.739130
Local,0.978261


In [81]:
#Calculate descriptives - std
ansAcc_participant.groupby('Type1').agg({'AnswerAccuracy':'std'})

,AnswerAccuracy
Type1,
Global,0.246962
Local,0.071221


In [18]:
# Inferentials

In [19]:
# Get a mean value for each participant
ansAcc_participant = answeringAccuracy(ansAccData,['participant','Type1', 'Type2'])

In [20]:
#control-flow
ansAcc_cflow_Global = ansAcc_participant.loc[(ansAcc_participant["Type1"]=='Global')]
ansAcc_cflow_Local = ansAcc_participant.loc[(ansAcc_participant["Type1"]=='Local')]
ansAccuracy_cflow_merge = ansAcc_cflow_Global.merge(ansAcc_cflow_Local, on=['participant','Type2'], suffixes=('_global', '_local'), how='inner')

print(len(ansAcc_cflow_Global), len(ansAcc_cflow_Local)) 
print(len(ansAccuracy_cflow_merge))

stats.wilcoxon(ansAccuracy_cflow_merge['AnswerAccuracy_global'], ansAccuracy_cflow_merge['AnswerAccuracy_local'],alternative='less')

46 46
46


C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=0.0, pvalue=6.108386720026067e-07)

In [ ]:
######################
#
# Comprehension efficiency (Response time)
#
######################

In [ ]:
#measure specific pre-processing

In [55]:
# select rows with 'questionID' is in the questions considered for the analysis
resTimeData = allData.loc[(allData['questionID'].isin([str(question["id"]) for question in questions]))].copy(deep=True)
    
#add question info
resTimeData = addQuestionInfo(resTimeData,questions)

#Select control-flow question type
resTimeData = resTimeData.loc[(resTimeData['Type2'] == 'Control-flow')] 

In [56]:
"""
2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
"""
resTimeData = resTimeData.drop(resTimeData[(resTimeData['participant'] == 'SP11-no') & (resTimeData['Type3'] == 'Exclusiveness')].index)
print(resTimeData.shape)

(732, 38)


In [57]:
#Example of query: time interval taken to answer each question by each participant
responseTime(resTimeData)

,questionID,participant,Type1,Type2,Type3,timeInterval
0,7,KP1-no,Local,Control-flow,Ordering,76380.0
1,7,KP10-no,Local,Control-flow,Ordering,29575.0
2,7,KP11-no,Local,Control-flow,Ordering,85422.0
3,7,KP12-no,Local,Control-flow,Ordering,39223.0
4,7,KP13-no,Local,Control-flow,Ordering,144094.0
...,...,...,...,...,...,...
361,28,SP5-no,Global,Control-flow,Repetition,111184.0
362,28,SP6-no,Global,Control-flow,Repetition,77086.0
363,28,SP7-no,Global,Control-flow,Repetition,129895.0
364,28,SP8-no,Global,Control-flow,Repetition,60026.0


In [111]:
#Check whether the response time is always longer for global than for local tasks for each participant on average
responseTimeCompare = responseTime(resTimeData).groupby(['participant','Type1','Type2'],as_index=False).agg({"timeInterval":"mean"})
responseTimeCompare_G = responseTimeCompare.loc[responseTimeCompare['Type1'] == 'Global']
responseTimeCompare_L = responseTimeCompare.loc[responseTimeCompare['Type1'] == 'Local']
responseTimeCompare_G_L = pd.merge(responseTimeCompare_G, responseTimeCompare_L, on='participant')

responseTimeCompare_G_L['Glo > Loc'] = responseTimeCompare_G_L.apply(lambda x: True if x['timeInterval_x'] > x['timeInterval_y'] else (x['timeInterval_x'],x['timeInterval_y']), axis=1)
responseTimeCompare_G_L

,participant,Type1_x,Type2_x,timeInterval_x,Type1_y,Type2_y,timeInterval_y,Glo > Loc
0,KP1-no,Global,Control-flow,62766.000000,Local,Control-flow,64573.000000,"(62766.0, 64573.0)"
1,KP10-no,Global,Control-flow,136581.000000,Local,Control-flow,38076.500000,True
2,KP11-no,Global,Control-flow,65067.750000,Local,Control-flow,101190.500000,"(65067.75, 101190.5)"
3,KP12-no,Global,Control-flow,109835.750000,Local,Control-flow,31047.750000,True
4,KP13-no,Global,Control-flow,196892.000000,Local,Control-flow,75692.000000,True
5,KP14-no,Global,Control-flow,195989.750000,Local,Control-flow,73249.250000,True
6,KP15-no,Global,Control-flow,222727.000000,Local,Control-flow,69701.250000,True
7,KP16-no,Global,Control-flow,89869.000000,Local,Control-flow,42855.250000,True
8,KP17-no,Global,Control-flow,179129.500000,Local,Control-flow,72315.500000,True
9,KP18-no,Global,Control-flow,152109.750000,Local,Control-flow,63441.500000,True


In [58]:
#Descriptives

In [59]:
#Calculate mean per participant
resTime_new_part = responseTime(resTimeData).groupby(['participant','Type1','Type2'],as_index=False).agg({"timeInterval":"mean"})
#Calculate descriptives - mean
resTime_new_part.groupby('Type1').agg({'timeInterval':'mean'})

,timeInterval
Type1,
Global,117592.971014
Local,54097.186594


In [82]:
#Calculate descriptives -std
resTime_new_part.groupby('Type1').agg({'timeInterval':'std'})

,timeInterval
Type1,
Global,41496.517509
Local,16829.302344


In [60]:
#Inferentials

In [61]:
resTime_new_part = responseTime(resTimeData).groupby(['participant','Type1','Type2'],as_index=False).agg({"timeInterval":"mean"})

In [62]:
#Control-flow
resTime_part_cflow_Global = resTime_new_part.loc[(resTime_new_part["Type1"]=='Global')]
resTime_part_cflow_Local = resTime_new_part.loc[(resTime_new_part["Type1"]=='Local')]
resTime_part_cflow_merge = resTime_part_cflow_Global.merge(resTime_part_cflow_Local, on=['participant','Type2'], suffixes=('_global', '_local'), how='inner')


print(len(resTime_part_cflow_Global), len(resTime_part_cflow_Local))
print(len(resTime_part_cflow_merge))

stats.wilcoxon(resTime_part_cflow_merge['timeInterval_global'], resTime_part_cflow_merge['timeInterval_local'],alternative='greater')

46 46
46


WilcoxonResult(statistic=1071.0, pvalue=6.110667527536862e-13)

In [63]:
########################
#
# Perceived difficulty
#
#######################

In [64]:
#measure specific pre-processing

In [65]:
# select rows with questionEventType='questionOffset' and 'questionID' is in the questions considered for the analysis
percDifficultyData =  allData.loc[(allData['questionEventType']=='questionOffset') & (allData['questionID'].isin([str(question["id"]+2) for question in questions]))].copy(deep=True)
# connect the perceived difficulty to the parent QuestionID (remember each task has three questions, main question, verbal justification, perceived diff question)
percDifficultyData['questionID'] = percDifficultyData['questionID'].astype('int') - 2

#add question info
percDifficultyData = addQuestionInfo(percDifficultyData,questions)

#Select control-flow question type
percDifficultyData = percDifficultyData.loc[(percDifficultyData['Type2'] == 'Control-flow')] 

In [66]:
#Example of query: for each participant, we obtain one aggregated measure for local questions and one aggregate measure for global questions
perceivedDifficulty(percDifficultyData,['participant','Type1'])

,participant,Type1,difficultyScore
0,KP1-no,Global,1.00
1,KP1-no,Local,1.00
2,KP10-no,Global,3.00
3,KP10-no,Local,0.75
4,KP11-no,Global,2.75
...,...,...,...
87,SP7-no,Local,0.00
88,SP8-no,Global,1.50
89,SP8-no,Local,0.50
90,SP9-no,Global,2.50


In [112]:
#Check whether perceived difficulty is always higher for global than for local tasks for each participant on average
perceivedDifficultyCompare = perceivedDifficulty(percDifficultyData,['participant','Type1'])
perceivedDifficultyCompare_G = perceivedDifficultyCompare.loc[perceivedDifficultyCompare['Type1'] == 'Global']
perceivedDifficultyCompare_L = perceivedDifficultyCompare.loc[perceivedDifficultyCompare['Type1'] == 'Local']
perceivedDifficultyCompare_G_L = pd.merge(perceivedDifficultyCompare_G, perceivedDifficultyCompare_L, on='participant')

perceivedDifficultyCompare_G_L['Glo > Loc'] = perceivedDifficultyCompare_G_L.apply(lambda x: True if x['difficultyScore_x'] > x['difficultyScore_y'] else (x['difficultyScore_x'],x['difficultyScore_y']), axis=1)
perceivedDifficultyCompare_G_L

,participant,Type1_x,difficultyScore_x,Type1_y,difficultyScore_y,Glo > Loc
0,KP1-no,Global,1.00,Local,1.00,"(1.0, 1.0)"
1,KP10-no,Global,3.00,Local,0.75,True
2,KP11-no,Global,2.75,Local,1.25,True
3,KP12-no,Global,1.25,Local,0.00,True
4,KP13-no,Global,3.00,Local,0.75,True
5,KP14-no,Global,2.00,Local,0.75,True
6,KP15-no,Global,2.25,Local,0.25,True
7,KP16-no,Global,1.75,Local,0.25,True
8,KP17-no,Global,3.75,Local,1.75,True
9,KP18-no,Global,2.25,Local,0.25,True


In [67]:
#Descriptives

In [68]:
#Calculate mean per participant
percDiff_part = perceivedDifficulty(percDifficultyData,['participant','Type1','Type2'])
#Calculate descriptives
percDiff_part.groupby('Type1').agg({'difficultyScore':'mean'})

,difficultyScore
Type1,
Global,1.891304
Local,0.619565


In [72]:
percDiff_part.groupby('Type1').agg({'difficultyScore':'std'})

,difficultyScore
Type1,
Global,0.772176
Local,0.521054


In [69]:
#Inferential

In [70]:
percDiff_part = perceivedDifficulty(percDifficultyData,['participant','Type1','Type2'])

In [71]:
percDiff_ControlFlow_Global = percDiff_part.loc[(percDiff_part["Type1"]=='Global')]
percDiff_ControlFlow_Local = percDiff_part.loc[(percDiff_part["Type1"]=='Local')]
percDiff_ControlFlow_merge = percDiff_ControlFlow_Global.merge(percDiff_ControlFlow_Local, on=['participant','Type2'], suffixes=('_global', '_local'), how='inner')

print(len(percDiff_ControlFlow_Global), len(percDiff_ControlFlow_Local))
print(len(percDiff_ControlFlow_merge))

stats.wilcoxon(percDiff_ControlFlow_merge['difficultyScore_global'],percDiff_ControlFlow_merge['difficultyScore_local'],alternative='greater')



46 46
46


C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=946.0, pvalue=4.983743900200151e-09)